# 成年人死亡率预测

<br>
<hr>

## 1. 实验介绍

### 1.1 实验背景

成年人死亡率指的是每一千人中15岁至60岁死亡的概率（数学期望）。这里我们给出了世界卫生组织（WHO）下属的全球卫生观察站（GHO）数据存储库跟踪的所有国家健康状况以及许多其他相关因素。要求利用训练数据建立回归模型，并预测成年人死亡率（**Adult Mortality**）。

### 1.2 实验要求

1. 训练数据包含`2336`条记录和`22`个字段，对训练数据进行一定的可视化数据分析（章节2.2）
2. 利用训练数据，选择合适的信息作为特征建立回归模型，并预测测试数据成年人死亡率
3. 利用`MO`平台进行模型性能评估

### 1.3 实验环境
可以使用基于 Python 的 `Pandas` 库进行数据相关处理，使用 `Sklearn` 库进行相关模型构建。


### 1.4 注意事项
1. 推荐使用基于 Python 的`Sklearn`库进行相关实验
2. 数据中可能会有一些字段的值存在缺失

## 2. 实验部分

### 2.1 导入相关包

In [3]:
import pandas as pd
import sklearn
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import joblib


### 2.2 数据读取和可视化分析

In [2]:
# 读取数据集
train_data = pd.read_csv('./data/train_data.csv')


In [ ]:
train_data


可以看到训练数据包含2336条记录和22个字段，每个字段含义说明如下：
1. Country：国家
2. Year：年份
3. Status：发达国家或发展中国家
4. Life expectancy：预期寿命
5. Infant deaths：每千人口中的婴儿死亡人数
6. Alcohol：人均酒精消费量（以升纯酒精为单位）
7. percentage expenditure：卫生支出占人均国内生产总值的百分比
8. Hepatitis B：一岁儿童乙型肝炎免疫疫苗接种率
9. Measles：麻疹每1000人报告的病例数
10. BMI：所有人群平均BMI指数
11. under-five deaths：每千人口中五岁以下死亡人数
12. Polio：1岁儿童脊髓灰质炎免疫覆盖率（%）
13. Total expenditure：政府卫生支出占政府总支出的百分比
14. Diphtheria：1岁儿童白喉、破伤风类毒素和百日咳免疫接种率（%）
15. HIV/AIDS：每千名活产婴儿死于艾滋病毒/艾滋病（0-4岁）
16. GDP：人均国内生产总值（美元）
17. Population：人口
18. thinness 1-19 years：10至19岁儿童和青少年的消瘦流行率
19. thinness 5-9 years：5至9岁儿童中的消瘦流行率
20. Income composition of resources：财力收入构成方面的人类发展指数（从0到1）
21. Schooling：受教育年限
22. Adult Mortality：成人死亡率（每1000人中15至60岁死亡的概率）

其中`Adult Mortality`字段为要预测的标签(`label`)

In [ ]:
# 查看数据集大小
train_data.shape


In [ ]:
# 计算各个特征之间的皮尔森相关系数
column_name = ['Year', 'Life expectancy ', 'infant deaths', 'Alcohol',
           'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ',
           'Polio', 'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
           ' thinness  1-19 years', ' thinness 5-9 years', 'Income composition of resources',
           'Schooling']
train_data[column_name].corr()


In [ ]:
# 将相关性矩阵绘制成热力图
corr = train_data[column_name].corr()
corr.style.background_gradient(cmap='coolwarm')


In [ ]:
# 利用seaborn检查可视化数据之间的依赖关系
import seaborn as sns
sns.pairplot(train_data[column_name])


### 2.3 模型拟合和成年人死亡率预测

要求同学们利用训练数据来建立回归模型，我们已经给出了一个可运行的基础代码，同学们可以对此进行参考

In [4]:
train_data = pd.read_csv('./data/train_data.csv')
model_filename = './model.pkl'
imputer_filename = './imputer.pkl'
scaler_filename = './scaler.pkl'


In [10]:
def preprocess_data(data, imputer=None, scaler=None):

    column_name = ['Year', 'Life expectancy ', 'infant deaths', 'Alcohol',
               'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ',
               'Polio', 'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
               ' thinness  1-19 years', ' thinness 5-9 years', 'Income composition of resources',
               'Schooling']
    data = data.drop(["Country", "Status"], axis=1)

    if imputer==None:
        imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
        imputer = imputer.fit(data[column_name])
    data[column_name] = imputer.transform(data[column_name])

    if scaler==None:
        scaler = MinMaxScaler()
        scaler = scaler.fit(data)
    data_norm = pd.DataFrame(scaler.transform(data), columns=data.columns)

    data_norm = data_norm.drop(['Year'], axis = 1)

    return data_norm, imputer, scaler


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

def model_fit(train_data):

    train_y = train_data.iloc[:,-1].values
    train_data = train_data.drop(["Adult Mortality"], axis=1)
    train_data_norm, imputer, scaler = preprocess_data(train_data)

    train_x = train_data_norm.values

    # regressor = LinearRegression()
    # regressor.fit(train_x, train_y)


    regressor = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    }
    grid_search = GridSearchCV(estimator=regressor,
                               param_grid=param_grid,
                               cv=5,
                               scoring='neg_mean_squared_error',
                               n_jobs=-1)
    
    grid_search.fit(train_x, train_y)
    best_regressor = grid_search.best_estimator_

    joblib.dump(best_regressor, model_filename)
    joblib.dump(imputer, imputer_filename)
    joblib.dump(scaler, scaler_filename)

    return regressor


In [8]:
def predict(test_data, filename):
    loaded_model = joblib.load(model_filename)
    imputer = joblib.load(imputer_filename)
    scaler = joblib.load(scaler_filename)

    test_data_norm, _, _ = preprocess_data(test_data, imputer, scaler)
    test_x = test_data_norm.values
    predictions = loaded_model.predict(test_x)

    return predictions


In [10]:
model = model_fit(train_data)


In [ ]:
# 打印模型的截距
print(model.intercept_)


In [ ]:
# 打印模型的斜率
print(model.coef_)


### 2.4 模型性能评估

我们可以看一下模型在训练集上的表现：

In [11]:
label = train_data.loc[:,'Adult Mortality']
data = train_data.iloc[:,:-1]
y_pred = predict(data, './model.pkl')
r2 = r2_score(label, y_pred)
mse = mean_squared_error(label, y_pred)
print("MSE is {}".format(mse))
print("R2 score is {}".format(r2))


MSE is 897.040289182684
R2 score is 0.9423002923058583


## 3. 作业提交

**作业要求及注意事项**：    
             
1. 点击左侧栏提交作业后点击【生成文件】则需要勾选与预测 predict() 函数的 cell相关的其它cell ，并将其转化成为 main.py 文件。                       
2. 请导入必要的包和第三方库以及该模型所依赖的 py 文件 (包括此文件中曾经导入过的)。             
3. 请加载你认为训练最佳的模型，即请按要求填写模型路径。              
4. `predict()` 函数的输入输出及函数名称请不要改动。
5. 提交的作业包括【程序报告.pdf】和对应的ipynb文件

===========================================  **模型预测代码答题区域**  ===========================================  
在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [6]:
import pandas as pd
import sklearn
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import joblib

model_filename = './model.pkl'
imputer_filename = './imputer.pkl'
scaler_filename = './scaler.pkl'

def preprocess_data(data, imputer=None, scaler=None):

# -------------------------- 请调整你的数据预处理过程 ---------------------------
## 输入：
#### data 为 pandas.DataFrame类型数据
#### imputer 为缺失值填充方式
#### scaler 为数据归一化方式
## 输出：
#### data_norm 为处理后的数据，为 pandas.DataFrame类型数据

    column_name = ['Year', 'Life expectancy ', 'infant deaths', 'Alcohol',
               'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ',
               'Polio', 'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
               ' thinness  1-19 years', ' thinness 5-9 years', 'Income composition of resources',
               'Schooling']
    data = data.drop(["Country", "Status"], axis=1)

    if imputer==None:
        imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
        imputer = imputer.fit(data[column_name])
    data[column_name] = imputer.transform(data[column_name])

    if scaler==None:
        scaler = MinMaxScaler()
        scaler = scaler.fit(data)
    data_norm = pd.DataFrame(scaler.transform(data), columns=data.columns)

    data_norm = data_norm.drop(['Year'], axis = 1)

    return data_norm

def predict(test_data):

# -------------------------- 请加载您最满意的模型 ---------------------------
# 加载模型(请加载你认为的最佳模型)
# 加载模型,加载请注意 filename 是相对路径, 与当前文件同级。
# test_data 为 pandas.DataFrame类型数据
    loaded_model = joblib.load(model_filename)
    imputer = joblib.load(imputer_filename)
    scaler = joblib.load(scaler_filename)

    test_data_norm = preprocess_data(test_data, imputer, scaler)
    test_x = test_data_norm.values
    predictions = loaded_model.predict(test_x)

    return predictions
